In [2]:
!pip install av

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 30.9/30.9 MB 26.2 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [34]:
import av
import time
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import os

In [22]:
# Source: https://stackoverflow.com/a/73998721

def with_cv2(video: str):
    """
    Link: https://pypi.org/project/opencv-python/
    My comments:
        I don't know why, but the last 4 or 5 timestamps are equal to 0 when they should not.
        Also, cv2 is slow. It took my computer 132 seconds to process the video.


    Parameters:
        video (str): Video path
    Returns:
        List of timestamps in ms
    """
    timestamps = []
    cap = cv2.VideoCapture(video)

    while cap.isOpened():
        frame_exists, curr_frame = cap.read()
        if frame_exists:
            timestamps.append(round(cap.get(cv2.CAP_PROP_POS_MSEC)))
        else:
            break

    cap.release()

    return timestamps

def with_pyav(video: str, index: int = 0):
    """
    Link: https://pypi.org/project/av/
    My comments:
        Works really well, but it is slower than ffprobe.

The big advantage is that ffmpeg does not have to be installed on the computer, because pyav installs it automatically

    Parameters:
        video (str): Video path
        index (int): Stream index of the video.
    Returns:
        List of timestamps in ms
    """
    container = av.open(video)
    video = container.streams.get(index)[0]

    if video.type != "video":
            raise ValueError(
                f'The index {index} is not a video stream. It is an {video.type} stream.'
            )

    av_timestamps = [
        int(packet.pts * video.time_base * 1000) for packet in container.demux(video) if packet.pts is not None
    ]

    container.close()
    av_timestamps.sort()

    return av_timestamps

In [23]:
_URL = './samples/video_sample.mp4'
_VIDEO_INDEX = 1

start = time.process_time()
cv2_timestamps = with_cv2(_URL)
print(f"With cv2 {time.process_time() - start} seconds")
print(f'Records found:', len(cv2_timestamps))
print([ft+25 for ft in cv2_timestamps[:50]])

start = time.process_time()
av_timestamps = with_pyav(_URL, index=_VIDEO_INDEX)
print(f"With av {time.process_time() - start} seconds")
print(f'Records found:', len(av_timestamps))
print(av_timestamps[:50])

With cv2 2.078125 seconds
Records found: 2403
[25, 67, 108, 136, 178, 206, 233, 275, 303, 344, 386, 400, 442, 469, 511, 539, 567, 608, 636, 678, 705, 733, 775, 803, 831, 872, 900, 942, 969, 1011, 1039, 1067, 1108, 1136, 1178, 1205, 1233, 1275, 1303, 1344, 1372, 1400, 1442, 1469, 1511, 1539, 1567, 1608, 1636, 1678]
With av 0.03125 seconds
Records found: 2403
[25, 66, 108, 136, 177, 205, 233, 275, 302, 344, 386, 400, 441, 469, 511, 539, 566, 608, 636, 677, 705, 733, 775, 802, 830, 872, 900, 941, 969, 1011, 1039, 1066, 1108, 1136, 1177, 1205, 1233, 1275, 1302, 1344, 1372, 1400, 1441, 1469, 1511, 1538, 1566, 1608, 1636, 1677]


# TO-DOs:

- Modify `deep-sort-app.py` to accept videos. Use the CV2 package for this. Even though it takes a while, it's compatible with frame extraction measures. It should output a DataFrame where each row contains 1) the timestamp since the start of the video, 2) the frame number, 4) the number of obstacles detected, and 4) the percentage of which is occupied detected obstacles.

In [44]:
def create_dirs(d:str, remove_existing:bool=True):
    Path(d).mkdir(parents=True, exist_ok=True)
    if remove_existing:
        for filename in os.listdir(d):
            filepath = os.path.join(d, filename)
            try:
                if os.path.isfile(filepath) or os.path.islink(filepath):
                    os.unlink(filepath)
            except Exception as e:
                print("Failed to delete %s. Reason: %s" % (filepath, e))

def extract_frames(video:str, output_dir:str):
    # Output: both images and a dataframe linking images to timestamps.
    
    # Create output directory in files
    create_dirs(output_dir)
    frames_dir = os.path.join(output_dir, 'frames/')
    create_dirs(frames_dir)
    
    timestamps = []
    cap = cv2.VideoCapture(video)
    frame_count = 0
    
    while cap.isOpened():
        frame_exists, curr_frame = cap.read()
        if frame_exists:
            frame_count += 1
            file_savename = os.path.join(frames_dir, f'{frame_count}.png')
            file_dfname = f'./frames/{frame_count}.png'
            # Record frame data
            timestamps.append({
                'frame_count':frame_count,
                'width': round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                'height': round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                'timestamp_ms':round(cap.get(cv2.CAP_PROP_POS_MSEC)),
                'file':file_dfname
            })
            # Save image
            cv2.imwrite(file_savename, curr_frame)
        else:
            break

    # Close video stream
    cap.release()
    
    # Record DataFrame
    df = pd.DataFrame(timestamps)
    
    # Save DataFrame
    df.to_csv(os.path.join(output_dir, 'timestamps.csv'), index=False)
    
    # Return
    return timestamps

In [45]:
ts = extract_frames(_URL, './samples/video_sample_timestamps/')